<a href="https://colab.research.google.com/github/FarrelAD/Hology-8-2025-Data-Mining-PRIVATE/blob/main/notebooks/vidi/nb_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Crowd Counting Model - SimpleCountingNet

This notebook implements a hybrid crowd counting model combining density map prediction and direct count regression.

**Features:**
- Dual-head architecture with density and count outputs
- Advanced data augmentation with Albumentations
- Hybrid loss function combining MSE and L1 losses
- Comprehensive evaluation and visualization
- Early stopping and model checkpointing
- Test set prediction and submission generation

# Import Libraries

In [ ]:
# Import Required Libraries and Setup
import os
import sys
import json
import warnings
from typing import Dict, List, Tuple, Optional
import zipfile

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import scipy.ndimage
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

# Suppress warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

# Detect environment
def detect_environment():
    """Detect if running in Colab, Kaggle, or local environment"""
    if 'google.colab' in sys.modules:
        return 'colab'
    elif 'kaggle_secrets' in sys.modules or os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
        return 'kaggle'
    else:
        return 'local'

ENV = detect_environment()
print(f"🔍 Detected environment: {ENV.upper()}")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Dataset Download and Setup

In [ ]:
# Environment-specific dataset setup
def setup_dataset_paths(env: str) -> Dict[str, str]:
    """Setup dataset paths based on environment"""
    
    if env == 'colab':
        # Google Colab paths
        dataset_name = "penyisihan-hology-8-0-2025-data-mining"
        drive_path = "/content/drive/MyDrive/PROJECTS/Cognivio/Percobaan Hology 8 2025/dataset"
        local_path = "../../data"
        
        # Mount Google Drive
        from google.colab import drive
        drive.mount('/content/drive')
        
        # Setup Kaggle credentials
        if not os.path.exists("/root/.kaggle/kaggle.json"):
            print("📥 Setting up Kaggle credentials...")
            from google.colab import files
            uploaded = files.upload()
            
            for fn in uploaded.keys():
                print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
            
            !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
        
        # Download and setup dataset
        if not os.path.exists(local_path):
            print("📥 Setting up dataset in Colab...")
            
            zip_path = f"/content/{dataset_name}.zip"
            
            # Download if not exists
            if not os.path.exists(zip_path):
                print("Dataset not found locally, downloading...")
                !pip install -q kaggle
                !kaggle competitions download -c {dataset_name} -p /content
            else:
                print("Dataset already exists, skipping download.")
            
            # Extract to Google Drive (for backup)
            os.makedirs(drive_path, exist_ok=True)
            if not os.listdir(drive_path):
                print("Extracting dataset to Google Drive...")
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(drive_path)
                print(f"Dataset extracted to: {drive_path}")
            else:
                print(f"Dataset already extracted at: {drive_path}")
            
            # Copy to local storage for faster access
            print("Copying dataset to Colab local storage (/content)...")
            !cp -r "{drive_path}" "{local_path}"
            print(f"✅ Dataset copied to {local_path}")
        
        return {
            'img_dir': f"{local_path}/train/images",
            'label_dir': f"{local_path}/train/labels",
            'test_dir': f"{local_path}/test/images",
            'save_dir': "/content/drive/MyDrive/PROJECTS/Cognivio/models"
        }
    
    elif env == 'kaggle':
        # Kaggle paths
        return {
            'img_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/train/images",
            'label_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/train/labels",
            'test_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/test/images",
            'save_dir': "/kaggle/working"
        }
    
    else:  # local
        # Local paths
        base_path = "../../data"
        return {
            'img_dir': f"{base_path}/train/images",
            'label_dir': f"{base_path}/train/labels",
            'test_dir': f"{base_path}/test/images",
            'save_dir': "models"
        }

# Setup paths
paths = setup_dataset_paths(ENV)
print(f"📁 Dataset paths configured for {ENV}:")
for key, path in paths.items():
    exists = "✅" if os.path.exists(path) else "⚠️"
    print(f"   {key}: {path} {exists}")

# Create save directory
os.makedirs(paths['save_dir'], exist_ok=True)

# Training Configuration

In [ ]:
# Training Configuration
config = {
    # Data paths
    'img_dir': paths['img_dir'],
    'label_dir': paths['label_dir'],
    'test_dir': paths['test_dir'],
    'save_dir': paths['save_dir'],
    
    # Model parameters
    'batch_size': 8,
    'epochs': 50,
    'lr': 0.001,
    'weight_decay': 1e-4,
    
    # Training parameters
    'num_workers': 2,
    'pin_memory': True,
    'early_stop_patience': 10,
    
    # Loss function weights
    'density_weight': 1.0,
    'count_weight': 10.0,
    
    # Model saving
    'best_model_path': os.path.join(paths['save_dir'], 'best_hybrid_model.pth'),
    'submission_path': os.path.join(paths['save_dir'], 'hybrid_submission.csv'),
    'seed': 31
}

print("📋 Configuration loaded:")
for key, value in config.items():
    print(f"  {key}: {value}")
    

# Set random seed for reproducibility
def set_seed(seed: int = 31) -> None:
    """Set random seeds for reproducibility."""
    torch.manual_seed(seed)
    np.random.seed(seed)

set_seed(config['seed'])
print(f"🎲 Random seed set to {config['seed']}")

# Data Preprocessing and Analysis

In [ ]:
print(f"🔍 Analyzing dataset...")

def load_crowd_counts(label_dir: str) -> Dict[str, int]:
    """Load crowd counts from JSON label files."""
    crowd_counts = {}
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.json')]
    
    for label_file in label_files:
        img_name = label_file.replace('.json', '.jpg')
        label_path = os.path.join(label_dir, label_file)
        
        with open(label_path, 'r') as f:
            data = json.load(f)
            crowd_counts[img_name] = data.get('human_num', 0)
    
    return crowd_counts

def create_density_map(
    annotations: List[Tuple[int, int]],
    img_height: int,
    img_width: int,
    sigma: float = 15.0
) -> np.ndarray:
    """Create density map from point annotations using Gaussian kernels."""
    density_map = np.zeros((img_height, img_width), dtype=np.float32)
    
    for x, y in annotations:
        if 0 <= x < img_width and 0 <= y < img_height:
            density_map[y, x] = 1.0
    
    # Apply Gaussian filter to create density map
    density_map = scipy.ndimage.gaussian_filter(density_map, sigma=sigma)
    
    return density_map

# Load crowd counts
crowd_counts = load_crowd_counts(config['label_dir'])
print(f"📊 Loaded {len(crowd_counts)} crowd count labels")

# Analyze crowd count distribution
crowd_values = np.array(list(crowd_counts.values()))
print(f"\n📈 Crowd count statistics:")
print(f"Mean: {crowd_values.mean():.2f}")
print(f"Std: {crowd_values.std():.2f}")
print(f"Min: {crowd_values.min()}")
print(f"Max: {crowd_values.max()}")
print(f"Median: {np.median(crowd_values):.2f}")

# Data Augmentation Setup

In [ ]:
print("🎨 Setting up data augmentation...")

# Data augmentation transforms
train_transforms = A.Compose([
    A.LongestMaxSize(max_size=512),
    A.PadIfNeeded(min_height=512, min_width=512, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.RandomResizedCrop(height=512, width=512, size=(512, 512), scale=(0.7, 1.0), ratio=(0.75, 1.33)),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.7),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], additional_targets={'density': 'mask'}, is_check_shapes=False)

val_transforms = A.Compose([
    A.LongestMaxSize(max_size=512),
    A.PadIfNeeded(min_height=512, min_width=512, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], additional_targets={'density': 'mask'}, is_check_shapes=False)

test_transforms = A.Compose([
    A.LongestMaxSize(max_size=512),
    A.PadIfNeeded(min_height=512, min_width=512, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], is_check_shapes=False)

print("✅ Data augmentation transforms configured!")

# Dataset Implementation

In [ ]:
class HybridCrowdDataset(Dataset):
    """Dataset for hybrid density + count regression training."""
    
    def __init__(
        self,
        image_dir: str,
        crowd_counts: Dict[str, int],
        image_files: List[str],
        transform=None,
        use_density_maps: bool = True
    ) -> None:
        self.image_dir = image_dir
        self.crowd_counts = crowd_counts
        self.image_files = [f for f in image_files if f in crowd_counts]
        self.transform = transform
        self.use_density_maps = use_density_maps
    
    def __len__(self) -> int:
        return len(self.image_files)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        
        # Load image (BGR → RGB)
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Count label
        count = self.crowd_counts[img_name]
        img_height, img_width = image.shape[:2]
        
        # Create simple density map (uniform distribution)
        target_height = img_height // 4
        target_width = img_width // 4
        density_map = np.full(
            (target_height, target_width),
            count / (target_height * target_width),
            dtype=np.float32
        )
        
        # Apply Albumentations transforms
        if self.transform:
            augmented = self.transform(image=image)
            augmented_image = augmented["image"]  # Tensor [C, H, W]
        else:
            augmented_image = ToTensorV2()(image=image)["image"]
        
        # Resize density map to match augmented image resolution (quarter size)
        aug_img_height, aug_img_width = augmented_image.shape[1:]  # (C, H, W)
        target_density_height = aug_img_height // 4
        target_density_width = aug_img_width // 4
        
        density_map_resized = cv2.resize(
            density_map,
            (target_density_width, target_density_height),
            interpolation=cv2.INTER_LINEAR
        )
        
        # Normalize to preserve total count after resize
        if density_map_resized.sum() > 0:
            density_map_resized *= (count / density_map_resized.sum())
        
        density_tensor = torch.tensor(density_map_resized, dtype=torch.float32).unsqueeze(0)
        count_tensor = torch.tensor(count, dtype=torch.float32)
        
        if self.use_density_maps:
            return augmented_image, density_tensor, count_tensor
        else:
            return augmented_image, count_tensor

print("✅ HybridCrowdDataset class implemented!")

# Data Loading and Preparation

In [ ]:
print("🔍 Preparing datasets...")

# Create train-val split
image_files = list(crowd_counts.keys())
train_files, val_files = train_test_split(
    image_files,
    test_size=0.2,
    random_state=42
)

# Create datasets
train_dataset = HybridCrowdDataset(
    image_dir=config['img_dir'],
    crowd_counts=crowd_counts,
    image_files=train_files,
    transform=train_transforms
)
val_dataset = HybridCrowdDataset(
    image_dir=config['img_dir'],
    crowd_counts=crowd_counts,
    image_files=val_files,
    transform=val_transforms
)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=config['num_workers'],
    pin_memory=config['pin_memory']
)
val_loader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=config['num_workers'],
    pin_memory=config['pin_memory']
)

print(f"📦 Dataset sizes:")
print(f"   Training set: {len(train_dataset)} images")
print(f"   Validation set: {len(val_dataset)} images")
print(f"🚀 Data loaders created!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")

# Model Architecture

In [ ]:
class SimpleCountingNet(nn.Module):
    """Simple CNN-based crowd counting model with dual outputs."""
    
    def __init__(self) -> None:
        super(SimpleCountingNet, self).__init__()
        
        # Simple CNN backbone
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        
        # Density regression head
        self.density_head = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, 1),
            nn.ReLU()
        )
        
        # Count regression head
        self.count_head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.ReLU()
        )
    
    def forward(
        self, 
        x: torch.Tensor, 
        true_density_size: Optional[Tuple[int, int]] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        # Extract features
        features = self.features(x)
        
        # Density map prediction
        density = self.density_head(features)
        
        # Upsample to match the size of the true density map
        if true_density_size is not None:
            density = F.interpolate(
                density,
                size=true_density_size,
                mode='bilinear',
                align_corners=False
            )
        else:  # Default upsampling if size is not provided (e.g., for inference)
            density = F.interpolate(
                density,
                scale_factor=8,  # Assuming feature map is 1/8th of original size
                mode='bilinear',
                align_corners=False
            )
        
        # Count prediction
        count = self.count_head(features).squeeze()
        
        return density, count

print("✅ SimpleCountingNet model architecture implemented!")

# Test model instantiation
model_test = SimpleCountingNet().to(device)
total_params = sum(p.numel() for p in model_test.parameters())
trainable_params = sum(p.numel() for p in model_test.parameters() if p.requires_grad)

print(f"📊 Model parameters:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")

del model_test

# Loss Functions and Training Setup

In [ ]:
class HybridLoss(nn.Module):
    """Hybrid loss combining density map MSE and count regression."""
    
    def __init__(
        self,
        density_weight: float = 1.0,
        count_weight: float = 10.0
    ) -> None:
        super(HybridLoss, self).__init__()
        self.density_weight = density_weight
        self.count_weight = count_weight
        self.mse_loss = nn.MSELoss()
        self.l1_loss = nn.L1Loss()
    
    def forward(
        self,
        pred_density: torch.Tensor,
        pred_count: torch.Tensor,
        true_density: torch.Tensor,
        true_count: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # Density map loss (MSE)
        density_loss = self.mse_loss(pred_density, true_density)
        
        # Count regression loss (L1)
        count_loss = self.l1_loss(pred_count, true_count)
        
        # Combined loss
        total_loss = self.density_weight * density_loss + self.count_weight * count_loss
        
        return total_loss, density_loss, count_loss

def train_hybrid_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    num_epochs: int = 50,
    learning_rate: float = 0.001
) -> Dict[str, List[float]]:
    """Train the hybrid model with both density and count supervision."""
    
    criterion = HybridLoss(
        density_weight=config['density_weight'],
        count_weight=config['count_weight']
    )
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=learning_rate,
        weight_decay=config['weight_decay']
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=5
    )
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_density_loss': [], 'val_density_loss': [],
        'train_count_loss': [], 'val_count_loss': [],
        'train_mae': [], 'val_mae': []
    }
    
    best_val_mae = float('inf')
    patience_counter = 0
    
    print("🚀 Starting hybrid training...")
    print("=" * 60)
    
    for epoch in range(num_epochs):
        print(f"\n📈 Epoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_density_loss = 0.0
        train_count_loss = 0.0
        train_mae = 0.0
        
        for batch_idx, (images, density_maps, counts) in enumerate(train_loader):
            images = images.to(device)
            density_maps = density_maps.to(device)
            counts = counts.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            pred_density, pred_count = model(
                images, 
                true_density_size=(density_maps.shape[2], density_maps.shape[3])
            )
            
            # Calculate loss
            total_loss, density_loss, count_loss = criterion(
                pred_density, pred_count, density_maps, counts
            )
            
            # Backward pass
            total_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            # Accumulate losses
            train_loss += total_loss.item()
            train_density_loss += density_loss.item()
            train_count_loss += count_loss.item()
            train_mae += nn.L1Loss()(pred_count, counts).item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_density_loss = 0.0
        val_count_loss = 0.0
        val_mae = 0.0
        
        with torch.no_grad():
            for images, density_maps, counts in val_loader:
                images = images.to(device)
                density_maps = density_maps.to(device)
                counts = counts.to(device)
                
                pred_density, pred_count = model(
                    images,
                    true_density_size=(density_maps.shape[2], density_maps.shape[3])
                )
                total_loss, density_loss, count_loss = criterion(
                    pred_density, pred_count, density_maps, counts
                )
                
                val_loss += total_loss.item()
                val_density_loss += density_loss.item()
                val_count_loss += count_loss.item()
                val_mae += nn.L1Loss()(pred_count, counts).item()
        
        # Calculate averages
        train_loss /= len(train_loader)
        train_density_loss /= len(train_loader)
        train_count_loss /= len(train_loader)
        train_mae /= len(train_loader)
        
        val_loss /= len(val_loader)
        val_density_loss /= len(val_loader)
        val_count_loss /= len(val_loader)
        val_mae /= len(val_loader)
        
        # Update learning rate
        scheduler.step(val_mae)
        
        # Store history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_density_loss'].append(train_density_loss)
        history['val_density_loss'].append(val_density_loss)
        history['train_count_loss'].append(train_count_loss)
        history['val_count_loss'].append(val_count_loss)
        history['train_mae'].append(train_mae)
        history['val_mae'].append(val_mae)
        
        # Print progress
        print(f'📊 Loss: {train_loss:.4f}/{val_loss:.4f} | '
              f'MAE: {train_mae:.4f}/{val_mae:.4f} | '
              f'Count Loss: {train_count_loss:.4f}/{val_count_loss:.4f}')
        print(f'🎯 Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
        
        # Early stopping
        if val_mae < best_val_mae:
            best_val_mae = val_mae
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), config['best_model_path'])
            print(f"✅ New best model saved! MAE: {best_val_mae:.4f}")
        else:
            patience_counter += 1
            print(f"⏳ No improvement for {patience_counter} epoch(s)")
        
        if patience_counter >= config['early_stop_patience']:
            print(f'🛑 Early stopping at epoch {epoch+1}')
            break
    
    # Load best model
    model.load_state_dict(torch.load(config['best_model_path']))
    print(f"\n🎉 Training completed! Best validation MAE: {best_val_mae:.4f}")
    
    return history

print("✅ Loss functions and training setup implemented!")

# Training Loop

In [ ]:
# Initialize model and training components
print("🏗️  Initializing model and training components...")

# Create model
model = SimpleCountingNet().to(device)
print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# Train the model
training_history = train_hybrid_model(
    model,
    train_loader,
    val_loader,
    num_epochs=config['epochs'],
    learning_rate=config['lr']
)

# Evaluation and Performance Metrics

In [ ]:
def evaluate_hybrid_model(
    model: nn.Module,
    data_loader: DataLoader,
    dataset_name: str = "Dataset"
) -> Dict[str, any]:
    """Comprehensive evaluation of hybrid model."""
    model.eval()
    
    all_pred_counts = []
    all_true_counts = []
    all_pred_densities = []
    all_true_densities = []
    
    with torch.no_grad():
        for images, density_maps, counts in data_loader:
            images = images.to(device)
            density_maps = density_maps.to(device)
            counts = counts.to(device)
            
            pred_density, pred_count = model(images)
            
            all_pred_counts.extend(pred_count.cpu().numpy())
            all_true_counts.extend(counts.cpu().numpy())
            all_pred_densities.extend(pred_density.cpu().numpy())
            all_true_densities.extend(density_maps.cpu().numpy())
    
    pred_counts = np.array(all_pred_counts)
    true_counts = np.array(all_true_counts)
    
    # Calculate metrics
    mae = mean_absolute_error(true_counts, pred_counts)
    mse = mean_squared_error(true_counts, pred_counts)
    rmse = np.sqrt(mse)
    
    # Additional metrics
    mape = np.mean(np.abs((true_counts - pred_counts) / (true_counts + 1e-8))) * 100
    correlation = np.corrcoef(true_counts, pred_counts)[0, 1]
    
    print(f"\n📊 {dataset_name} Evaluation Results:")
    print("=" * 40)
    print(f"MAE (Count):     {mae:.4f}")
    print(f"MSE (Count):     {mse:.4f}")
    print(f"RMSE (Count):    {rmse:.4f}")
    print(f"MAPE (%):        {mape:.2f}")
    print(f"Correlation:     {correlation:.4f}")
    print(f"Mean True Count: {np.mean(true_counts):.2f}")
    print(f"Mean Pred Count: {np.mean(pred_counts):.2f}")
    
    return {
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'mape': mape,
        'correlation': correlation,
        'pred_counts': pred_counts,
        'true_counts': true_counts
    }

# Evaluate on both sets
print("🎯 Evaluating model performance...")
train_results = evaluate_hybrid_model(model, train_loader, "Training Set")
val_results = evaluate_hybrid_model(model, val_loader, "Validation Set")

# Results Visualization and Analysis

In [ ]:
def plot_hybrid_training_history(history: Dict[str, List[float]]) -> None:
    """Plot training history for hybrid model."""
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Total loss
    axes[0, 0].plot(history['train_loss'], label='Train Loss', color='blue')
    axes[0, 0].plot(history['val_loss'], label='Val Loss', color='red')
    axes[0, 0].set_title('Total Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Count loss
    axes[0, 1].plot(history['train_count_loss'], label='Train Count Loss', color='blue')
    axes[0, 1].plot(history['val_count_loss'], label='Val Count Loss', color='red')
    axes[0, 1].set_title('Count Regression Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Density loss
    axes[0, 2].plot(history['train_density_loss'], label='Train Density Loss', color='blue')
    axes[0, 2].plot(history['val_density_loss'], label='Val Density Loss', color='red')
    axes[0, 2].set_title('Density Map Loss')
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('Loss')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # MAE
    axes[1, 0].plot(history['train_mae'], label='Train MAE', color='blue')
    axes[1, 0].plot(history['val_mae'], label='Val MAE', color='red')
    axes[1, 0].set_title('Mean Absolute Error')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('MAE')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Predictions vs targets
    axes[1, 1].scatter(val_results['true_counts'], val_results['pred_counts'], alpha=0.6)
    axes[1, 1].plot([0, max(val_results['true_counts'])], [0, max(val_results['true_counts'])], 'r--')
    axes[1, 1].set_xlabel('True Count')
    axes[1, 1].set_ylabel('Predicted Count')
    axes[1, 1].set_title('Validation: Predictions vs True')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Error distribution
    errors = val_results['pred_counts'] - val_results['true_counts']
    axes[1, 2].hist(errors, bins=20, alpha=0.7, edgecolor='black')
    axes[1, 2].axvline(x=0, color='red', linestyle='--')
    axes[1, 2].set_xlabel('Prediction Error')
    axes[1, 2].set_ylabel('Frequency')
    axes[1, 2].set_title('Error Distribution')
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot training curves
print("📈 Plotting training history...")
plot_hybrid_training_history(training_history)

# Model Visualization

In [ ]:
def visualize_hybrid_predictions(
    model: nn.Module,
    dataset: Dataset,
    num_samples: int = 8
) -> None:
    """Visualize density maps and count predictions."""
    model.eval()
    
    # Get sample data
    indices = np.random.choice(len(dataset), min(num_samples//2, len(dataset)), replace=False)
    
    fig, axes = plt.subplots(len(indices), 4, figsize=(16, 4*len(indices)))
    if len(indices) == 1:
        axes = axes.reshape(1, -1)
    
    with torch.no_grad():
        for i, idx in enumerate(indices):
            image, true_density, true_count = dataset[idx]
            image_batch = image.unsqueeze(0).to(device)
            
            # Get predicted density and count
            pred_density, pred_count = model(
                image_batch, 
                true_density_size=(true_density.shape[1], true_density.shape[2])
            )
            
            # Denormalize image for visualization
            img_viz = image.clone()
            img_viz = img_viz * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1) + torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
            img_viz = torch.clamp(img_viz, 0, 1).permute(1, 2, 0).numpy()
            
            # Original image
            axes[i, 0].imshow(img_viz)
            axes[i, 0].set_title(f'Original\nTrue: {true_count:.0f}, Pred: {pred_count.item():.1f}')
            axes[i, 0].axis('off')
            
            # True density map (squeeze channel dim)
            true_density_viz = true_density.squeeze().cpu().numpy()
            axes[i, 1].imshow(true_density_viz, cmap='hot')
            axes[i, 1].set_title('True Density')
            axes[i, 1].axis('off')
            
            # Predicted density map (squeeze channel dim)
            pred_density_viz = pred_density.squeeze().cpu().numpy()
            axes[i, 2].imshow(pred_density_viz, cmap='hot')
            axes[i, 2].set_title('Predicted Density')
            axes[i, 2].axis('off')
            
            # Density difference
            diff = pred_density_viz - true_density_viz
            im = axes[i, 3].imshow(diff, cmap='RdBu', vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)))
            axes[i, 3].set_title('Difference')
            axes[i, 3].axis('off')
            
            # Add colorbar for the first sample
            if i == 0:
                fig.colorbar(im, ax=axes[i, 3])
    
    plt.suptitle('Hybrid Model Predictions: Density Maps + Counts', fontsize=16)
    plt.tight_layout()
    plt.show()

# Visualize sample predictions
print("🎨 Visualizing sample predictions...")
visualize_hybrid_predictions(model, val_dataset, num_samples=8)

# Test Prediction and Generate Submission

In [ ]:
class TestDataset(Dataset):
    """Dataset for test images."""
    
    def __init__(self, image_dir: str, transform=None) -> None:
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = sorted(
            [f for f in os.listdir(image_dir) if f.endswith('.jpg')],
            key=lambda x: int(os.path.splitext(x)[0])
        )
    
    def __len__(self) -> int:
        return len(self.image_files)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, str]:
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, img_name

def generate_hybrid_test_predictions(
    model: nn.Module,
    test_dir: str,
    output_file: str = 'hybrid_submission.csv'
) -> pd.DataFrame:
    """Generate test predictions using the hybrid model."""
    print("📝 Generating test predictions with hybrid model...")
    
    test_dataset = TestDataset(test_dir, test_transforms)
    test_loader = DataLoader(
        test_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=2
    )
    
    model.eval()
    predictions = []
    image_names = []
    
    with torch.no_grad():
        for images, names in test_loader:
            images = images.to(device)
            pred_density, pred_count = model(images)
            
            # Use count predictions for submission
            batch_preds = pred_count.cpu().numpy()
            predictions.extend([max(0, int(round(pred))) for pred in batch_preds])
            image_names.extend(names)
    
    # Create submission DataFrame
    submission_df = pd.DataFrame({
        'image_id': image_names,
        'predicted_count': predictions
    })
    
    # Sort by image_id
    submission_df['sort_key'] = submission_df['image_id'].apply(lambda x: int(os.path.splitext(x)[0]))
    submission_df = submission_df.sort_values('sort_key').drop('sort_key', axis=1).reset_index(drop=True)
    
    # Save submission
    submission_df.to_csv(output_file, index=False)
    
    print(f"✅ Submission saved to {output_file}")
    print(f"📊 Predictions for {len(submission_df)} test images")
    
    # Statistics
    pred_counts = submission_df['predicted_count'].values
    print(f"\n📈 Test Predictions Statistics:")
    print(f"Min: {pred_counts.min()}")
    print(f"Max: {pred_counts.max()}")
    print(f"Mean: {pred_counts.mean():.2f}")
    print(f"Median: {np.median(pred_counts):.2f}")
    
    return submission_df

# Generate test predictions
submission_df = generate_hybrid_test_predictions(model, config['test_dir'], config['submission_path'])

# Display first few rows
print("\n📋 Sample submission:")
print(submission_df.head(10))

# Plot prediction distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(submission_df['predicted_count'], bins=30, alpha=0.7, edgecolor='black')
plt.title('Distribution of Test Predictions')
plt.xlabel('Predicted Count')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(submission_df['predicted_count'])
plt.title('Test Predictions Box Plot')
plt.ylabel('Predicted Count')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Model Saving and Final Summary

In [ ]:
# Model Saving for Competition Submission
print("💾 Saving trained model...")

# Save complete model (architecture + weights)
model_complete_path = os.path.join(config['save_dir'], 'hybrid_counting_model_complete.pth')
torch.save(model, model_complete_path)
print(f"✅ Complete model saved to: {model_complete_path}")

# Save model state dict only
model_weights_path = os.path.join(config['save_dir'], 'hybrid_counting_model_weights.pth')
torch.save(model.state_dict(), model_weights_path)
print(f"✅ Model weights saved to: {model_weights_path}")

# Final Summary
print("\n" + "="*60)
print("🎉 HYBRID CROWD COUNTING MODEL - FINAL SUMMARY")
print("="*60)

print(f"\n🏗️  Model Architecture: SimpleCountingNet")
print(f"- CNN Backbone: Custom 4-layer CNN")
print(f"- Dual outputs: Density maps + Direct count regression")
print(f"- Total parameters: {sum(p.numel() for p in model.parameters()):,}")

print(f"\n📋 Training Configuration:")
print(f"- Training images: {len(train_dataset)}")
print(f"- Validation images: {len(val_dataset)}")
print(f"- Batch size: {config['batch_size']}")
print(f"- Loss function: Hybrid (Density MSE + Count L1)")

print(f"\n📊 Final Performance Metrics:")
print(f"- Training MAE: {train_results['mae']:.4f}")
print(f"- Validation MAE: {val_results['mae']:.4f}")
print(f"- Validation RMSE: {val_results['rmse']:.4f}")
print(f"- Validation Correlation: {val_results['correlation']:.4f}")

print(f"\n🎯 Key Advantages of Hybrid Approach:")
print(f"- Combines spatial density information with global count regression")
print(f"- Density maps provide spatial understanding")
print(f"- Count regression ensures accurate total predictions")
print(f"- Mutual supervision improves both predictions")

print(f"\n📝 Test Set Predictions:")
print(f"- Generated predictions for {len(submission_df)} test images")
print(f"- Mean predicted count: {submission_df['predicted_count'].mean():.2f}")
print(f"- Prediction range: {submission_df['predicted_count'].min()} to {submission_df['predicted_count'].max()}")

print(f"\n🚀 Next Steps for Improvement:")
print(f"- Implement proper point annotations for better density maps")
print(f"- Add attention mechanisms to focus on crowd regions")
print(f"- Experiment with different loss function weights")
print(f"- Use pretrained backbones (ResNet, EfficientNet)")
print(f"- Implement multi-scale training and testing")

print("\n" + "="*60)
print("✅ Training and evaluation complete!")
print(f"🎯 Model saved at: {config['save_dir']}")
print("📊 Evaluation results and visualizations have been generated above.")